# Clustering Crypto

In [116]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [117]:
# Load the crypto_data.csv dataset.
data_file = "crypto_data.csv"
crypto_df = pd.read_csv(data_file)
crypto_df.head()
crypto_df['IsTrading'].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [118]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df['IsTrading'].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [119]:
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [120]:
crypto_df['Algorithm'].isnull().sum()

0

In [121]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].value_counts()

Scrypt                   394
X11                      182
SHA-256                  121
X13                       54
PoS                       42
                        ... 
VeChainThor Authority      1
Ouroboros                  1
POS 2.0                    1
Proof-of-BibleHash         1
TRC10                      1
Name: Algorithm, Length: 89, dtype: int64

In [122]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns='IsTrading')
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [123]:
crypto_df.isnull().sum()

Unnamed: 0           0
CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [124]:
crypto_df.isna().sum()

Unnamed: 0           0
CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [125]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna(how='any')
crypto_df.isnull().sum()

Unnamed: 0         0
CoinName           0
Algorithm          0
IsTrading          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [126]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000
10,DASH,Dash,X11,True,PoW/PoS,9.031294e+06,22000000
11,XMR,Monero,CryptoNight-V7,True,PoW,1.720114e+07,0
12,ETC,Ethereum Classic,Ethash,True,PoW,1.133597e+08,210000000
13,ZEC,ZCash,Equihash,True,PoW,7.383056e+06,21000000


In [127]:
crypto_df = crypto_df.set_index('Unnamed: 0')
crypto_df.index.name = None
crypto_df.index

Index(['42', '404', '1337', 'BTC', 'ETH', 'LTC', 'DASH', 'XMR', 'ETC', 'ZEC',
       ...
       'TCH', 'WAVES', 'PART', 'BTT', 'NXT', 'ZEPH', 'GAP', 'BDX', 'ZEN',
       'XBC'],
      dtype='object', length=532)

In [128]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = crypto_df["CoinName"]
crypto_names_df.head()

42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [129]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName', 'IsTrading'])
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [130]:
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [131]:
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype('float64')
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [132]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [135]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.33282857,  0.98520171, -0.59330137],
       [-0.31616041,  0.98473291, -0.59364669],
       [ 2.30280038,  1.48817097, -0.63757476],
       ...,
       [ 0.3241705 , -2.23123909,  0.41237608],
       [-0.15439161, -1.94632862,  0.51726083],
       [-0.28801469,  0.70772271, -0.23832425]])

In [136]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
crypto_pca_df = pd.DataFrame(
    data=X_pca, columns =['PC1', 'PC2', 'PC3']
)
crypto_pca_df.head()

,PC1,PC2,PC3
0,-0.332829,0.985202,-0.593301
1,-0.316160,0.984733,-0.593647
2,2.302800,1.488171,-0.637575
3,-0.146417,-1.252772,0.192939
4,-0.157067,-2.009781,0.390042


In [139]:
crypto_pca_df.index = crypto_df.index
crypto_pca_df.head()

,PC1,PC2,PC3
42,-0.332829,0.985202,-0.593301
404,-0.316160,0.984733,-0.593647
1337,2.302800,1.488171,-0.637575
BTC,-0.146417,-1.252772,0.192939
ETH,-0.157067,-2.009781,0.390042


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [141]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

# Calculate the inertia for the range of K values

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)
    
# Create the elbow curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [142]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)
predictions

array([0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2,
       2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0,
       0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2,
       0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0,
       2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0,
       0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0,
       0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2,
       0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0,

In [153]:
crypto_df.head()
crypto_pca_df.head()

,PC1,PC2,PC3
42,-0.332829,0.985202,-0.593301
404,-0.316160,0.984733,-0.593647
1337,2.302800,1.488171,-0.637575
BTC,-0.146417,-1.252772,0.192939
ETH,-0.157067,-2.009781,0.390042


In [154]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, crypto_pca_df], axis=1, join="inner")
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.332829,0.985202,-0.593301,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.316160,0.984733,-0.593647,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.302800,1.488171,-0.637575,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.146417,-1.252772,0.192939,Bitcoin,2
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.157067,-2.009781,0.390042,Ethereum,2
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.164607,-1.092876,-0.023888,Litecoin,2
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.395236,1.249886,-0.533732,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.162508,-2.014521,0.471540,Monero,2
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.155508,-2.009953,0.390026,Ethereum Classic,2
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.154391,-1.946329,0.517261,ZCash,2


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [164]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    hover_name='CoinName',
    hover_data=['Algorithm'],
    width=800,
)
fig.show()

In [165]:
# Create a table with tradable cryptocurrencies.
crypto_table = clustered_df.hvplot.table()
crypto_table

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class]

In [19]:
# Print the total number of tradable cryptocurrencies.
num_currencies = clustered_df['CoinName'].count()
print(f"There are {num_currencies} tradable cryptocurrencies")

There are 532 tradable cryptocurrencies.


In [174]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler=MinMaxScaler()
scaled_coins = scaler.fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
scaled_coins

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [180]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(data=scaled_coins, columns=['TotalCoinSupply', 'TotalCoinsMined'])
scaled_df.index = clustered_df.index
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df['Class'] = clustered_df['Class']

scaled_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,2
ETH,0.000000e+00,0.000109,Ethereum,2
LTC,8.400000e-05,0.000064,Litecoin,2
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,2
ETC,2.100000e-04,0.000115,Ethereum Classic,2
ZEC,2.100000e-05,0.000007,ZCash,2


In [181]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols='CoinName')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)